In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.integrate import solve_ivp  # numerical integration solver

Helper functions:

In [ ]:
def mse(x, y, theta_hat):
  """Compute the mean squared error

  Args:
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding
      measurement values to the inputs.
    theta_hat (float): An estimate of the slope parameter

  Returns:
    float: The mean squared error of the data with the estimated parameter.
  """

  # Compute the estimated y
  y_hat = theta_hat * x

  # Compute mean squared error
  mse = np.mean((y - y_hat)**2)

  return mse


def solve_normal_eqn(x, y):
  """Solve the normal equations to produce the value of theta_hat that minimizes
    MSE.

    Args:
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding
      measurement values to the inputs.

  Returns:
    float: the value for theta_hat arrived from minimizing MSE
  """

  # Compute theta_hat analytically
  theta_hat = (x.T @ y) / (x.T @ x)

  return theta_hat


def likelihood(theta_hat, x, y):
  """The likelihood function for a linear model with noise sampled from a
    Gaussian distribution with zero mean and unit variance.

  Args:
    theta_hat (float): An estimate of the slope parameter.
    x (ndarray): An array of shape (samples,) that contains the input values.
    y (ndarray): An array of shape (samples,) that contains the corresponding
      measurement values to the inputs.

  Returns:
    float: the likelihood value for the theta_hat estimate
  """
  sigma = 1

  # Compute Gaussian likelihood
  pdf = 1 / np.sqrt(2 * np.pi * sigma**2) * np.exp(-(y - theta_hat * x)**2 / (2 * sigma**2))

  return pdf

# Likelihood vs probability:
# L(θ|x,y)=p(y|x,θ)
# p(y|x,θ) → “probability of observing the response y given parameter θ and input x”
# L(θ|x,y) → “likelihood model that parameters θ produced response y from input x”

def integrate_exponential(x0, a, dt, T, y):
  """Compute solution of the differential equation xdot=x0*(1-x0)*np.sum(a*delta_payoff) with
  initial condition x0 for a duration T. Use time step dt for numerical
  solution.

  Args:
    x0 (scalar): initial condition (x at time 0)
    a (array): parameter of xdot
    delta_payoff(array): parameter of xdot
    dt (scalar): timestep of the simulation
    T (scalar): total duration of the simulation

  Returns:
    ndarray, ndarray: `x` for all simulation steps and the time `t` at each step
  """

  # Initialize variables
  t = np.arange(0, T, dt)
  x = np.zeros_like(t, dtype=complex)
  x[0] = x0 # This is x at time t_0

  payoff_A = alpha * y # activation payoff; alpha - activation propensity; x - activation level
  payoff_I = iota * (1-y) # inactivation payoff; iota - inactivation propensity; (1-x) - inactivation level

  delta_payoff = payoff_A - payoff_I
  # Step through system and integrate in time
  for k in range(1, len(t)):

    # for each point in time, compute xdot from x[k-1]
    xdot = x[k-1]*(1-x[k-1])*np.sum(a*delta_payoff)

    # Update x based on x[k-1] and xdot
    x[k] = x[k-1] +  xdot * dt

  return x, t





Implement the dynamical system from An Evolutionary Game Theory Model of Spontaneous Brain Functioning DOI:10.1038/s41598-017-15865-w.\

In [ ]:
# Choose parameters
alpha, iota = 1, 1

T = 10      # total Time duration
dt = 0.001  # timestep of our simulation
x0 = 0.2     # initial condition of x at time 0

a = 5
y = 4

# Use Euler's method
x, t = integrate_exponential(x0, a, dt, T, y)

# Visualize
plt.plot(t, x)
plt.xlabel('Time (s)')
plt.ylabel('x')
plt.show()